In [1]:
### Quantiacs Trading System Template
# This program may take more than 10 minutes
# import necessary Packages
import quantiacsToolbox
import numpy as np
from sklearn import svm
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#initials for testing the code

#reading file 
F_AD = pd.read_csv(r'/home/anshul/Desktop/Quantiacs/tickerData/F_AD.txt')

#getting features
CLOSE = np.array(F_AD.loc[:252-1, [' CLOSE']])
#plt.plot(CLOSE)
VOL = np.array(F_AD.loc[:252-1, [' VOL']])
#plt.plot(VOL)
OI = np.array(F_AD.loc[:252-1, [' VOL']])
#plt.plot(OI)
HIGH = np.array(F_AD.loc[:252-1, [' HIGH']])
#plt.plot(HIGH)
LOW = np.array(F_AD.loc[:252-1, [' LOW']])
#plt.plot(LOW)
OPEN = np.array(F_AD.loc[:252-1, [' OPEN']])

In [3]:
#code
lookback = CLOSE.shape[0]
gap=5
X = np.concatenate([CLOSE[i:i + gap] for i in range(lookback - gap)], axis=1).T
y = np.sign((CLOSE[gap:lookback] - CLOSE[gap - 1:lookback - 1]).T[0])
#print(y[y==0])
clf = svm.SVC()
clf.fit(X, y)
print(clf.predict(CLOSE[-gap:].T))
#plt.plot(y)
plt.plot(clf.predict(CLOSE[-gap:].T))

[1.]


In [34]:
#code -> vwap/close
#20 days
vol = np.nan_to_num(VOL[-20:]).reshape(-1,1)
close = np.nan_to_num(CLOSE[-20:]).reshape(-1,1)
vwap = np.sum(np.multiply(vol,close))/np.sum(vol)
alpha = vwap/close[-1]
print(alpha)
mean = np.mean(vwap/close)


if alpha > 1:
    pos=1
elif alpha < 0.3:
    pos=-1
else:
    pos=0

[0.99579135]


In [20]:
a = np.array([1,2,3,4]).reshape(-1,1)
b = np.array([1,2,3,4]).reshape(-1,1)
print(a,b)
print(np.multiply(a,b))

[[1]
 [2]
 [3]
 [4]] [[1]
 [2]
 [3]
 [4]]
[[ 1]
 [ 4]
 [ 9]
 [16]]


In [13]:
class myStrategy(object):
    def myTradingSystem(self,DATE, OPEN, HIGH, LOW, CLOSE, VOL, OI, P, R, RINFO, exposure, equity, settings):
        
        nMarkets = len(settings['markets'])
        dur = settings['duration']
        
        vec_vwap = np.zeros((1, nMarkets), dtype='float')
        pos = np.zeros((1, nMarkets), dtype='float')
        for i in range(nMarkets):
            close = np.nan_to_num(CLOSE[-dur:, i]).reshape(-1,1)
            vol = np.nan_to_num(VOL[-dur:, i]).reshape(-1,1)
            vwap = np.sum(np.multiply(vol,close))/np.sum(vol)
            alpha = np.nan_to_num(vwap/close[-1])
            vec_vwap[0, i] = alpha
        #print(vec_vwap)
        mean = np.mean(vec_vwap)
        #print('mean:', mean)
        for i in range(nMarkets):
            if vec_vwap[0,i] >= 1:
                pos[0, i]=-1
            elif vec_vwap[0,i] <= 1:
                pos[0, i]=1
            else:
                pos[0, i]=0


        return pos, settings


    def mySettings(self):
        """ Define your trading system settings here """

        settings = {}

        # Futures Contracts
#         settings['markets'] = ['CASH', 'F_AD', 'F_AE', 'F_AH', 'F_AX', 'F_BC', 'F_BG', 'F_BO', 'F_BP', 'F_C',  'F_CA',
#                                'F_CC', 'F_CD', 'F_CF', 'F_CL', 'F_CT', 'F_DL', 'F_DM', 'F_DT', 'F_DX', 'F_DZ', 'F_EB',
#                                'F_EC', 'F_ED', 'F_ES', 'F_F',  'F_FB', 'F_FC', 'F_FL', 'F_FM', 'F_FP', 'F_FV', 'F_FY',
#                                'F_GC', 'F_GD', 'F_GS', 'F_GX', 'F_HG', 'F_HO', 'F_HP', 'F_JY', 'F_KC', 'F_LB', 'F_LC',
#                                'F_LN', 'F_LQ', 'F_LR', 'F_LU', 'F_LX', 'F_MD', 'F_MP', 'F_ND', 'F_NG', 'F_NQ', 'F_NR',
#                                'F_NY', 'F_O',  'F_OJ', 'F_PA', 'F_PL', 'F_PQ', 'F_RB', 'F_RF', 'F_RP', 'F_RR', 'F_RU',
#                                'F_RY', 'F_S',  'F_SB', 'F_SF', 'F_SH', 'F_SI', 'F_SM', 'F_SS', 'F_SX', 'F_TR', 'F_TU',
#                                'F_TY', 'F_UB', 'F_US', 'F_UZ', 'F_VF', 'F_VT', 'F_VW', 'F_VX',  'F_W', 'F_XX', 'F_YM',
#                               'F_ZQ']
        #markets to  decide:
        #currency -> 'US-Dollar Index', 'euro_fx', 'jap_yen', 'safrica rand', 
        #agri -> 'corn', 'cotton', coffee, sugar, wheat, rice, 
        #index -> volatility index, bloomberg commodity index, russel200, dax, cac 
        #energy -> nat gas,brent crude, newcastle coal,
        #int rate -> 30 day fed, eurodollar
        #bond-> 2 yr tre, 10 yr, 30 yr, euro bond
        #metal -> gold, cu, 
        #stocks -> NA
        settings['markets'] = ['F_DX', 'F_EC', 'F_JY', 'F_TR', 
                                'F_C', 'F_CT', 'F_KC', 'F_SB', 'F_W',  'F_NR', 
                                'F_VX', 'F_AH', 'F_RU', 'F_AX', 'F_CA', 
                                'F_NG', 'F_BC', 'F_LQ',
                                'F_ZQ', 'F_ED', 
                                'F_TU', 'F_TY', 'F_US', 'F_DT',  
                                'F_GC', 'F_HG']
        settings['lookback'] = 252
        settings['budget'] = 10 ** 6
        settings['slippage'] = 0.05

        settings['duration'] = 20

        return settings

In [14]:
result = quantiacsToolbox.runts(myStrategy)

copying data from cache
Evaluating Trading System <class '__main__.myStrategy'>


/home/anshul/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':
